In [6]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
os.chdir('/Users/rhinomonkey/Desktop/Project1_Machine Learning/')
df_ac = pd.read_csv('accident.csv')
df_per = pd.read_csv('person.csv')

FileNotFoundError: File b'accident.csv' does not exist

In [ ]:
df_ac['HARM_EV'].value_counts().head()
harm_ev= {12: 'SameRoadVehicle', 8: 'Pedestrian', 1: 'OverTurn', 42: 'Trees', 
          33: 'Curb', 34: 'Ditch', 35: 'Embankment'}
df_ac['harm_ev']=df_ac['HARM_EV'].apply(lambda x: harm_ev[x] 
if (x==12 or x==8 or x==1 or x==42 or x==33 or x==34 or x== 35)  else 'Other')

In [ ]:
df_ml = pd.DataFrame()
df_ml['STATE']=df_ac['STATE'].fillna('TX')
df_ml['MONTH']=df_ac['MONTH'].fillna(0)
df_ml['DAY_WEEK']=df_ac['DAY_WEEK'].fillna(6)
df_ml['DAY']=df_ac['DAY'].fillna(3)
df_ml['HOUR']=df_ac['HOUR'].fillna(18)
df_ml['harm_ev']=df_ac['harm_ev'].fillna('Embankment')
df_ml['DRUNK_DR']=df_ac['DRUNK_DR'].fillna(0)
df_ml['harm_ev'].unique()

In [ ]:
df_ml['harm_ev'] = df_ml['harm_ev'].replace(['Embankment', 'SameRoadVehicle'], ['EM', 'SRV'])
x=pd.get_dummies(df_ml['harm_ev'], prefix = 'harm_ev')
x = pd.concat([x, pd.get_dummies(df_ml['STATE'], prefix ='STATE')], axis=1)
#x = pd.concat([x, pd.get_dummies(df_ml['DRUNK_DR'], prefix ='DD')], axis=1)
x = pd.concat([x, pd.get_dummies(df_ml['MONTH'], prefix ='MONTH')], axis=1)
x = pd.concat([x, pd.get_dummies(df_ml['DAY_WEEK'], prefix ='Dw')], axis=1)
x = pd.concat([x, pd.get_dummies(df_ml['DAY'], prefix ='DAY')], axis=1)
x = pd.concat([x, pd.get_dummies(df_ml['HOUR'], prefix ='HOUR')], axis=1)
x.head(3)

In [ ]:
df_ml['DRUNK_DR']=df_ml['DRUNK_DR'].apply(lambda x: 0 if (x==0)  else 1)
df_ml['DRUNK_DR'].value_counts()

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
Y = le.fit_transform(df_ml['DRUNK_DR'])
x = scaler.fit_transform(x)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, 
                                                    random_state=62)

In [ ]:
#Base rate mode - which predict that everyone maximum fatal
def base_rate_model(x):
    y = np.zeros(x.shape[0])
    return y

In [ ]:
#How the accurate is my base rate model?
y_base_rate = base_rate_model(x_test)
from sklearn.metrics import accuracy_score
print('Base rate accuracy is %2.2f'% accuracy_score(y_test, y_base_rate))

In [2]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=1)

In [3]:
model.fit(x_train, y_train)

NameError: name 'x_train' is not defined

In [ ]:
print('Logistic accuracy is %2.2f'% accuracy_score(y_test, model.predict(x_test)))

In [ ]:
## c score to compare the model
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [ ]:
print('----Base Model-----')
#Base rate Auc
base_roc_auc = roc_auc_score(y_test, base_rate_model(x_test))
print('Base Rate AUC = %2.2f'%base_roc_auc)
print(classification_report(y_test, base_rate_model(x_test)))
print('\n\n-----Logistic Model-----')
#Logistic AUC
logist_roc_auc = roc_auc_score(y_test, model.predict(x_test))
print('logistic AUC = %2.2f'%logist_roc_auc)
print(classification_report(y_test,model.predict(x_test)))

In [4]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(x_test)[:,1])

NameError: name 'y_test' is not defined

In [5]:
#plot of a ROC Curve for a specific for a specific class
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % logist_roc_auc)
plt.plot([0,1],[0,1],'k--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.5])
plt.xlabel(['False Positive Rate'])
plt.ylabel(['True Positive Rate'])
plt.title('Receiver operating charateristic example')
plt.legend(loc ='lower right')
plt.show()

NameError: name 'fpr' is not defined